In [2]:
import pandas as pd
df = pd.read_csv('infiltration_01-03-2018.csv')

In [2]:
df.head()

,dst_port,protocol,timestamp,flow_duration,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,...,fwd_seg_size_min,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,0,0,01/03/2018 08:17:11,115307855,5,0,0,0,0,0,...,0,1812348.0,0.0,1812348,1812348,56700000.0,6.010058e+06,61000000,52500000,Benign
1,0,0,01/03/2018 08:20:07,60997457,2,0,0,0,0,0,...,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000,Benign
2,67,17,01/03/2018 08:17:18,61149019,5,0,1500,0,300,300,...,8,3530939.0,0.0,3530939,3530939,19200000.0,1.250000e+07,32600000,7999725,Benign
3,0,0,01/03/2018 08:22:09,60997555,2,0,0,0,0,0,...,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000,Benign
4,0,0,01/03/2018 08:24:11,61997503,3,0,0,0,0,0,...,0,999909.0,0.0,999909,999909,61000000.0,0.000000e+00,61000000,61000000,Benign


In [3]:
print("Read {} rows.".format(len(df)))

Read 331100 rows.


In [4]:
df.dropna(inplace=True,axis=1)

In [5]:
df['label'] = df['label'].apply(lambda x: 1 if x == 'Infilteration' else 0)

In [6]:
df = df.drop('timestamp', axis = 1)

In [7]:
import numpy as np
np.any(np.isnan(df))

False

In [8]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
def _assert_all_finite(df):
    df = np.asanyarray(df)
    if (df.dtype.char in np.typecodes['AllFloat'] and not np.isfinite(df.sum())
            and not np.isfinite(df).all()):
        raise ValueError("Input contains NaN, infinity"
                         " or a value too large for %r." % df.dtype)
df.fillna(999, inplace=True)

In [9]:
np.all(np.isfinite(df))

True

In [10]:
pd.set_option('display.max_columns', 5)
pd.set_option('display.max_rows', 5)
# display 5 rows
display(df[0:5])

,dst_port,protocol,...,idle_min,label
0,0,0,...,52500000,0
1,0,0,...,61000000,0
2,67,17,...,7999725,0
3,0,0,...,61000000,0
4,0,0,...,61000000,0


In [11]:
import os
from sklearn import metrics
from scipy.stats import zscore

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [12]:
analyze(df)


331100 rows
** dst_port:16149 (4%)
** protocol:[6:64.3%,17:33.54%,0:2.16%]
** flow_duration:153887 (46%)
** tot_fwd_pkts:328 (0%)
** tot_bwd_pkts:408 (0%)
** totlen_fwd_pkts:3953 (1%)
** totlen_bwd_pkts:8805 (2%)
** fwd_pkt_len_max:1340 (0%)
** fwd_pkt_len_min:126 (0%)
** fwd_pkt_len_mean:10070 (3%)
** fwd_pkt_len_std:13857 (4%)
** bwd_pkt_len_max:857 (0%)
** bwd_pkt_len_min:273 (0%)
** bwd_pkt_len_mean:13136 (3%)
** bwd_pkt_len_std:14800 (4%)
** flow_pkts_s:155521 (46%)
** flow_iat_mean:146408 (44%)
** flow_iat_std:127942 (38%)
** flow_iat_max:83986 (25%)
** flow_iat_min:24730 (7%)
** fwd_iat_tot:96006 (28%)
** fwd_iat_mean:130082 (39%)
** fwd_iat_std:97961 (29%)
** fwd_iat_max:95023 (28%)
** fwd_iat_min:42617 (12%)
** bwd_iat_tot:69546 (21%)
** bwd_iat_mean:100353 (30%)
** bwd_iat_std:85205 (25%)
** bwd_iat_max:52382 (15%)
** bwd_iat_min:50729 (15%)
** fwd_psh_flags:[0:93.53%,1:6.47%]
** bwd_psh_flags:[0:100.0%]
** fwd_urg_flags:[0:99.72%,1:0.28%]
** bwd_urg_flags:[0:100.0%]
** fwd_

In [13]:
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [14]:
df.columns

Index(['dst_port', 'protocol', 'flow_duration', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'flow_pkts_s', 'flow_iat_mean', 'flow_iat_std',
       'flow_iat_max', 'flow_iat_min', 'fwd_iat_tot', 'fwd_iat_mean',
       'fwd_iat_std', 'fwd_iat_max', 'fwd_iat_min', 'bwd_iat_tot',
       'bwd_iat_mean', 'bwd_iat_std', 'bwd_iat_max', 'bwd_iat_min',
       'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags', 'bwd_urg_flags',
       'fwd_header_len', 'bwd_header_len', 'fwd_pkts_s', 'bwd_pkts_s',
       'pkt_len_min', 'pkt_len_max', 'pkt_len_mean', 'pkt_len_std',
       'pkt_len_var', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
       'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt', 'cwe_flag_count',
       'ece_flag_cnt', 'down_up_ratio', 'pkt_size_avg', 'fwd_seg_size_avg',
       'bwd_se

In [15]:
df.dropna(inplace=True,axis=1)
df[0:5]
x_columns = df.columns.drop('label')
x = df[x_columns].values
dummies = pd.get_dummies(df['label']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

In [16]:
df.groupby('label')['label'].count()

label
0    238037
1     93063
Name: label, dtype: int64

In [24]:
df.to_csv("infiltered.csv")

In [17]:
import io
import requests
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping


x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto',
                           restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
          callbacks=[monitor],verbose=2,epochs=100)

D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 248325 samples, validate on 82775 samples
Epoch 1/100
248325/248325 - 8s - loss: 200.9774 - val_loss: 1.1361
Epoch 2/100
248325/248325 - 8s - loss: 0.7229 - val_loss: 0.6195
Epoch 3/100
248325/248325 - 8s - loss: 0.6003 - val_loss: 1.0212
Epoch 4/100
248325/248325 - 8s - loss: 0.5831 - val_loss: 0.6152
Epoch 5/100
248325/248325 - 8s - loss: 0.5685 - val_loss: 0.6280
Epoch 6/100
248325/248325 - 8s - loss: 0.5596 - val_loss: 0.5812
Epoch 7/100
248325/248325 - 8s - loss: 0.5763 - val_loss: 0.5800
Epoch 8/100
248325/248325 - 8s - loss: 0.5620 - val_loss: 0.5592
Epoch 9/100
248325/248325 - 8s - loss: 0.5782 - val_loss: 0.5702
Epoch 10/100
248325/248325 - 8s - loss: 0.5603 - val_loss: 0.5610
Epoch 11/100
248325/248325 - 8s - loss: 0.5602 - val_loss: 0

In [18]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.7547447900936273


In [8]:
df1 = pd.read_csv('infiltered.csv')
df1.drop(['protocol', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'flow_pkts_s', 'flow_iat_std',
       'flow_iat_max', 'fwd_iat_mean',
       'fwd_iat_std', 'fwd_iat_max', 'fwd_iat_min', 'bwd_iat_tot',
       'bwd_iat_mean', 'bwd_iat_std', 'bwd_iat_max', 'bwd_iat_min',
       'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags', 'bwd_urg_flags',
       'bwd_header_len', 'fwd_pkts_s',
       'pkt_len_min', 'pkt_len_max', 'pkt_len_mean', 'pkt_len_std',
       'pkt_len_var', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
       'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt', 'cwe_flag_count',
       'ece_flag_cnt', 'down_up_ratio', 'pkt_size_avg', 'fwd_seg_size_avg',
       'bwd_seg_size_avg', 'fwd_byts_b_avg', 'fwd_pkts_b_avg',
       'fwd_blk_rate_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg',
       'bwd_blk_rate_avg', 'subflow_fwd_pkts', 'subflow_fwd_byts',
       'subflow_bwd_pkts', 'subflow_bwd_byts',
       'init_bwd_win_byts', 'fwd_act_data_pkts', 
       'active_std', 'active_max', 'active_min', 'idle_mean',
       'idle_std', 'idle_max', 'idle_min'],axis=1, inplace=True)

In [9]:
df1.columns

Index(['dst_port', 'flow_duration', 'flow_iat_mean', 'flow_iat_min',
       'fwd_iat_tot', 'fwd_header_len', 'bwd_pkts_s', 'init_fwd_win_byts',
       'fwd_seg_size_min', 'active_mean', 'label'],
      dtype='object')

In [10]:
df1.head()

,dst_port,flow_duration,flow_iat_mean,flow_iat_min,fwd_iat_tot,fwd_header_len,bwd_pkts_s,init_fwd_win_byts,fwd_seg_size_min,active_mean,label
0,0,115307855,28800000.0,812396,115000000,0,0.0,-1,0,1812348.0,0
1,0,60997457,61000000.0,61000000,61000000,0,0.0,-1,0,0.0,0
2,67,61149019,15300000.0,3530939,61100000,40,0.0,-1,8,3530939.0,0
3,0,60997555,61000000.0,61000000,61000000,0,0.0,-1,0,0.0,0
4,0,61997503,31000000.0,999909,62000000,0,0.0,-1,0,999909.0,0
